In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# necessary file installation details
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
# Import required libraries
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd

In [ ]:
# Load the multi_news dataset from Hugging Face and take only the 'test' split for efficiency
dataset = pd.read_csv('/content/drive/MyDrive/covid_abstracts.csv')

# Convert the test dataset to a pandas DataFrame and take only 2000 random samples
df.head()

,document,summary
4830,Tweet with a location \n \n You can add locati...,– Denis Finley has taken to Twitter to call Po...
1255,CNN host Piers Morgan just called to discuss h...,– CNN's Piers Morgan thinks gun-rights propone...
80,White House communications director Anthony Sc...,– New White House communications director Anth...
3044,CLOSE Scientists say they've found archaeologi...,– Scientists say they have the first physical ...
4486,Click image above to view graphic \n \n Althou...,– Scientists are calling it a breakthrough and...


In [ ]:
# Load a pre-trained sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
# Encode each summary in the DataFrame using the sentence transformer model and store the embeddings in a list
passage_embeddings = list(model.encode(df['summary'].to_list(), show_progress_bar=True))

# Print the shape of the first passage embedding
passage_embeddings[0].shape

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

(384,)

In [ ]:
# Declare a query string
query = "Covid preventions"

In [ ]:
# Encode the query using the sentence transformer model
query_embedding = model.encode(query, show_progress_bar=True)
query_embedding.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(384,)

In [ ]:
# Calculate the cosine similarity between the query embedding and the passage embeddings
similarities = util.cos_sim(query_embedding, passage_embeddings)
similarities.shape

torch.Size([1, 2000])

In [ ]:
# Find the indices of the top 3 most similar passages
top_indices = torch.topk(similarities.flatten(), 3).indices
top_indices

tensor([1279, 1926, 1928])

In [ ]:
top_relevant_passages = [df.iloc[x.item()]['summary'] + "..." for x in top_indices]
top_relevant_passages

['– Chris Christie may have a pretty irate nurse taking the Garden State to task over the execution of his mandatory-quarantine policy toward Ebola, but he\'s standing by it. "I think this is a policy that will become a national policy sooner rather than later," he told Fox News Sunday today, as per Politico. "It was my conclusion that we needed to do this to protect the public health of the people of New Jersey. We have taken this action, and I have no second thoughts about it." Fellow Republican Darrell Issa, meanwhile, says that New Jersey\'s quarantine—like those in Illinois and New York—is an attempt to compensate for the White House\'s lack of leadership on Ebola, but probably not the best course. "Science has told us that if someone doesn’t have an elevated temperature or the other later symptoms, that we can rely on them not being contagious," he said. "If that’s true, then immediate isolation of people for 21 days is not the answer. Again, trust matters." Elsewhere on your Sun